In [2]:
!pip uninstall -y langchain langchain-core langchain-community langchain-openai langchain-anthropic langchain_google_genai
!pip install -U langchain langchain-community langchain-core langchain-openai --quiet



Found existing installation: langchain 0.3.17
Uninstalling langchain-0.3.17:
  Successfully uninstalled langchain-0.3.17
Found existing installation: langchain-core 0.3.33
Uninstalling langchain-core-0.3.33:
  Successfully uninstalled langchain-core-0.3.33
Found existing installation: langchain-community 0.3.16
Uninstalling langchain-community-0.3.16:
  Successfully uninstalled langchain-community-0.3.16
Found existing installation: langchain-openai 0.3.3
Uninstalling langchain-openai-0.3.3:
  Successfully uninstalled langchain-openai-0.3.3


## Download new site

In [57]:
from langchain_community.document_loaders import SitemapLoader
import os
from urllib.parse import urlparse
from pathlib import Path

# Define paths
CACHE_DIR = "sitemap_data"
os.makedirs(CACHE_DIR, exist_ok=True)

def slugify_url(url: str) -> str:
    """Convert URL to a filesystem-safe name."""
    parsed = urlparse(url)
    return f"{parsed.netloc}{parsed.path}".replace("/", "_").replace(".", "_")

# Load sitemap and save to disk
sitemap_url = "https://news.itmo.ru/module/sitemap.php"
loader = SitemapLoader(sitemap_url)
documents = loader.load()

# Save each document as a separate file
for doc in documents:
    url = doc.metadata["source"]
    filename = f"{slugify_url(url)}.txt"
    filepath = Path(CACHE_DIR) / filename
    
    # Write content to disk
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(doc.page_content)
    
    print(f"Saved: {filepath}")


Exception ignored in: <finalize object at 0x29d1e0b00; dead>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/weakref.py", line 585, in __call__
    def __call__(self, _=None):

KeyboardInterrupt: 
Fetching pages:   0%|          | 291/59692 [00:19<1:07:07, 14.75it/s]


KeyboardInterrupt: 

## Prepare model and api keys

In [14]:
import os
import json
# Set API keys (replace with your actual keys)
TAVILY_API_KEY = "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"
OPENAI_API_KEY = "sk-aitunnel-GlP8MYqkbc3ethsARsD0KLI9LfiiG1kP"

os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY



## Defina ReAct

In [5]:
from agent.agent import process_query
input_question = "В каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в топ-400 мировых университетов?\n1. ARWU (Shanghai Ranking)\n2. Times Higher Education (THE) World University Rankings\n3. QS World University Rankings\n4. U.S. News & World Report Best Global Universities"

from datetime import datetime

start = datetime.now()
output = process_query(input_question, 1)
print(output)

print('time ', datetime.now()-start)


{"id": 1, "answer": 3, "reasoning": "ИТМО впервые вошел в топ-400 мировых университетов по рейтингу QS World University Rankings в 2021 году", "sources": ["https://topreytings.ru/itmo-reyting-vuzov-mira/", "https://assistent-system.ru/reyting/reyting-itmo-sredi-vuzov-rossii-2021/", "https://news.itmo.ru/ru/university_live/ratings/news/9494/"]}
time  0:00:13.806190


## Работа с запросом в формате АПИ

In [6]:
req = {
  "query": "В каком городе находится главный кампус Университета ИТМО?\n1. Москва\n2. Санкт-Петербург\n3. Екатеринбург\n4. Нижний Новгород",
  "id": 1
}

start = datetime.now()
output = process_query(req['query'], 1)
print(output)
print('time ', datetime.now()-start)


{"id": 1, "answer": 2, "reasoning": "главный кампус Университета ИТМО находится в Санкт-Петербурге", "sources": []}
time  0:00:07.999394


## Run multiple tests

**Вопросы, сгенеренные с помощью перплексити**

In [28]:
questions = [
    {
        "question": "В каком году был основан Университет ИТМО?\n1.1945 годl\n2.1917 год\n3.1900 год\n4.1980 год",
        "answer_num": 3
    },
    {
        "question": "Какое первоначальное название имело учебное заведение, предшествующее ИТМО?\n1.Ленинградский политехнический институт\n2.Механико-оптическое и часовое отделение\n3.Институт точной механики\n4.Институт оптики и фотоники",
        "answer_num": 2
    },
    {
        "question": "Когда университет получил статус национального исследовательского университета?\n1.В 2005 году\n2.В 2010 году\n3.В 2009 году\n4.В 2012 году",
        "answer_num": 3
    },
    {
        "question": "Какое здание стало современным главным зданием университета?\n1.Здание на Кронверкском проспекте\n2.Здание на Невском проспекте\n3.Здание на Московском проспекте\n4.Здание на Лиговском проспекте",
        "answer_num": 1
    },
    {
        "question": "Какой орден был награждён Ленинградский институт точной механики и оптики в 1980 году?\n1.Орден Почёта\n2.Орден Дружбы народов\n3.Орден Трудового Красного Знамени\n4.Орден Знак Почёта",
        "answer_num": 3
    },
    {
        "question": "Когда ИТМО получил статус университета?\n1.В 1994 году\n2.В 1990 году\n3.В 1996 году\n4.В 2000 году",
        "answer_num": 1
    },
    {
        "question": "Кто является бессменным ректором университета с 1996 года?\n1.Сергей Петров\n2.Александр Кулешов\n3.Игорь Кузнецов\n4.Владимир Васильев",
        "answer_num": 4
    },
    {
        "question": "Сколько корпусов и общежитий имеет университет на 2022 год?\n1.12 корпусов и 6 общежитий\n2.10 корпусов и 5 общежитий\n3.15 корпусов и 8 общежитий\n4.20 корпусов и 10 общежитий",
        "answer_num": 1
    },
    {
        "question": "Какое событие произошло в 1917 году, связанное с университетом?\n1.Отделение выделилось в самостоятельное учебное заведение\n2.Переименование института\n3.Открытие нового факультета\n4.Закрытие института",
        "answer_num": 1
    },
]

In [22]:
for question in questions:
    print('Question: ', question['question'])
    start = datetime.now()
    output = json.loads(process_query(question['question'], 1))
    answer = output['answer']
    print('Answer: ', answer)
    print('Reasoning:', output['reasoning'])
    print('Sources:', ", ".join(output['sources']))
    if answer==question['answer_num']:
        print('Corect!!!')
    else:
        print('Wrong :(')
    print('time ', datetime.now()-start)
    print('\n\n')


Question:  В каком году был основан Университет ИТМО?
1.1945 годl
2.1917 год
3.1900 год
4.1980 год
Answer:  3
Reasoning: Университет ИТМО был основан в 1900 году.
Sources: 
Corect!!!
time  0:00:05.914324



Question:  Какое первоначальное название имело учебное заведение, предшествующее ИТМО?
1.Ленинградский политехнический институт
2.Механико-оптическое и часовое отделение
3.Институт точной механики
4.Институт оптики и фотоники
Answer:  2
Reasoning: Механико-оптическое и часовое отделение
Sources: 
Corect!!!
time  0:00:06.074372



Question:  Когда университет получил статус национального исследовательского университета?
1.В 2005 году
2.В 2010 году
3.В 2009 году
4.В 2012 году
Answer:  3
Reasoning: Университет ИТМО получил статус национального исследовательского университета в 2009 году
Sources: https://russiaedu.ru/vuz/78/ifmo/, https://nanojournal.ifmo.ru/university_itmo/, https://ru.wikipedia.org/wiki/Университет_ИТМО
Corect!!!
time  0:00:10.263539



Question:  Какое здание стало с